Used to generate XGB predictions

In [1]:
from xgboost import XGBRegressor

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score, mean_squared_error

import seaborn as sns
import shap
from xgb_params import xgb_params as params, num_top_features as top_n

In [2]:
params

{'n_estimators': 200,
 'device': 'cuda',
 'objective': 'reg:squarederror',
 'eval_metric': 'rmsle',
 'enable_categorical': True,
 'early_stopping_rounds': 20,
 'colsample_bytree': 0.7,
 'eta': 0.1,
 'gamma': 0,
 'max_depth': 6,
 'min_child_weight': 3.0,
 'reg_alpha': 93.0,
 'reg_lambda': 0.8685796539747039,
 'n_jobs': 4}

In [3]:
# File Paths
test_df_path = '../../Dataset/test.csv/test.csv'
process_train_path = './processed/processed_train_2.csv'
processed_test_path = './processed/processed_test_2.csv'
feats_path = './best_feats/Onehot_only_mean_NANs_modified_prices.csv'
output_path = './output/xgb_pred.csv'


In [4]:
feats_df = pd.read_csv(feats_path)
feats = feats_df['col_name'].values.tolist()[:top_n]
processed_df = pd.read_csv(process_train_path)
processed_df = processed_df[feats+['price_doc']]
X = processed_df.drop(['price_doc'], axis=1)
y = processed_df['price_doc']
X = X[feats]

test_df = pd.read_csv(test_df_path)
processed_test_df = pd.read_csv(processed_test_path)
processed_test_df.head()

,id,full_sq,life_sq,floor,max_floor,build_year,num_room,kitch_sq,state,area_m,...,big_road1_1line_yes,railroad_1line_no,railroad_1line_yes,material_1.0,material_2.0,material_3.0,material_4.0,material_5.0,material_6.0,material_nan
0,30474,39.0,20.700000,2,9,1998.0,1,8.9,3.0,2.615514e+07,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,30475,79.2,34.404467,8,17,0.0,3,1.0,1.0,2.553630e+07,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,30476,40.5,25.100000,3,5,1960.0,2,4.8,2.0,9.946335e+06,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,30477,62.8,36.000000,17,17,2016.0,2,62.8,3.0,2.149409e+07,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,30478,40.0,40.000000,17,17,0.0,1,1.0,1.0,2.553630e+07,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
def train(X, y, evalset=None):    
    model = XGBRegressor(
                    # verbosity=0,
                    n_estimators=params['n_estimators'],
                    device=params['device'],
                    objective=params['objective'],
                    eval_metric=params['eval_metric'],
                    enable_categorical=params['enable_categorical'],
                    early_stopping_rounds=params['early_stopping_rounds'] if evalset else None,

                    eta=params['eta'],
                    max_depth = params['max_depth'],
                    gamma = params['gamma'],
                    # reg_alpha = params['reg_alpha'],
                    min_child_weight=params['min_child_weight'],
                    colsample_bytree=params['colsample_bytree'],
                    n_jobs=params['n_jobs']
                )
    if not evalset:
        model.fit(X, y, verbose=True)
    else:
        model.fit(X, y, eval_set=evalset, verbose=True)

    return model

def find_best_model(processed_df):
    # Return best model from kfolds
    best_model = None
    min_loss = float('inf')
    cv = KFold(n_splits=10, shuffle=True, random_state=42)
    for fold, (train_idx, test_idx) in enumerate(cv.split(processed_df)):
        X_train = processed_df.iloc[train_idx]
        y_train = X_train["price_doc"]
        X_train.drop(["price_doc"], axis=1, inplace=True)

        X_val = processed_df.iloc[test_idx]
        y_val = X_val["price_doc"]
        X_val.drop(["price_doc"], axis=1, inplace=True)

        evalset = [(X_val, y_val)]
        model = train(X_train, y_train, evalset)

        pred = model.predict(X_val)
        loss = mean_squared_error(y_val, pred)

        if loss < min_loss:
            min_loss = loss
            best_model = model
    
    return best_model

In [6]:
model = find_best_model(processed_df)
# model = train(X, y)
pred = model.predict(processed_test_df[feats])


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.Spars

[0]	validation_0-rmsle:0.61020
[1]	validation_0-rmsle:0.59041
[2]	validation_0-rmsle:0.57701
[3]	validation_0-rmsle:0.56338
[4]	validation_0-rmsle:0.55406
[5]	validation_0-rmsle:0.54600
[6]	validation_0-rmsle:0.53751
[7]	validation_0-rmsle:0.53171
[8]	validation_0-rmsle:0.52522
[9]	validation_0-rmsle:0.51941
[10]	validation_0-rmsle:0.51521
[11]	validation_0-rmsle:0.51100
[12]	validation_0-rmsle:0.50831
[13]	validation_0-rmsle:0.50476
[14]	validation_0-rmsle:0.50179
[15]	validation_0-rmsle:0.49960
[16]	validation_0-rmsle:0.49736
[17]	validation_0-rmsle:0.49599
[18]	validation_0-rmsle:0.49412
[19]	validation_0-rmsle:0.49271
[20]	validation_0-rmsle:0.49117
[21]	validation_0-rmsle:0.49049
[22]	validation_0-rmsle:0.48949
[23]	validation_0-rmsle:0.48853
[24]	validation_0-rmsle:0.48761
[25]	validation_0-rmsle:0.48673
[26]	validation_0-rmsle:0.48641
[27]	validation_0-rmsle:0.48587
[28]	validation_0-rmsle:0.48529
[29]	validation_0-rmsle:0.48472
[30]	validation_0-rmsle:0.48412
[31]	validation_0-

is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
[18:56:12] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0cec3277c4d9d0165-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.



[1]	validation_0-rmsle:0.57053
[2]	validation_0-rmsle:0.55688
[3]	validation_0-rmsle:0.54327
[4]	validation_0-rmsle:0.53357
[5]	validation_0-rmsle:0.52567
[6]	validation_0-rmsle:0.51631
[7]	validation_0-rmsle:0.50997
[8]	validation_0-rmsle:0.50290
[9]	validation_0-rmsle:0.49729
[10]	validation_0-rmsle:0.49310
[11]	validation_0-rmsle:0.48887
[12]	validation_0-rmsle:0.48573
[13]	validation_0-rmsle:0.48236
[14]	validation_0-rmsle:0.47952
[15]	validation_0-rmsle:0.47685
[16]	validation_0-rmsle:0.47463
[17]	validation_0-rmsle:0.47298
[18]	validation_0-rmsle:0.47145
[19]	validation_0-rmsle:0.46983
[20]	validation_0-rmsle:0.46864
[21]	validation_0-rmsle:0.46782
[22]	validation_0-rmsle:0.46670
[23]	validation_0-rmsle:0.46579
[24]	validation_0-rmsle:0.46524
[25]	validation_0-rmsle:0.46426
[26]	validation_0-rmsle:0.46372
[27]	validation_0-rmsle:0.46319
[28]	validation_0-rmsle:0.46246
[29]	validation_0-rmsle:0.46197
[30]	validation_0-rmsle:0.46151
[31]	validation_0-rmsle:0.46090
[32]	validation_0

is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.Spars

...

is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.Spars

[5]	validation_0-rmsle:0.56216
[6]	validation_0-rmsle:0.55302
[7]	validation_0-rmsle:0.54675
[8]	validation_0-rmsle:0.54012
[9]	validation_0-rmsle:0.53461
[10]	validation_0-rmsle:0.53041
[11]	validation_0-rmsle:0.52640
[12]	validation_0-rmsle:0.52365
[13]	validation_0-rmsle:0.52096
[14]	validation_0-rmsle:0.51800
[15]	validation_0-rmsle:0.51563
[16]	validation_0-rmsle:0.51354
[17]	validation_0-rmsle:0.51197
[18]	validation_0-rmsle:0.51051
[19]	validation_0-rmsle:0.50907
[20]	validation_0-rmsle:0.50804
[21]	validation_0-rmsle:0.50729
[22]	validation_0-rmsle:0.50628
[23]	validation_0-rmsle:0.50542
[24]	validation_0-rmsle:0.50485
[25]	validation_0-rmsle:0.50397
[26]	validation_0-rmsle:0.50318
[27]	validation_0-rmsle:0.50256
[28]	validation_0-rmsle:0.50205
[29]	validation_0-rmsle:0.50167
[30]	validation_0-rmsle:0.50125
[31]	validation_0-rmsle:0.50071
[32]	validation_0-rmsle:0.50058
[33]	validation_0-rmsle:0.50029
[34]	validation_0-rmsle:0.50010
[35]	validation_0-rmsle:0.49992
[36]	validati

is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.Spars

[5]	validation_0-rmsle:0.53656
[6]	validation_0-rmsle:0.52744
[7]	validation_0-rmsle:0.52126
[8]	validation_0-rmsle:0.51445
[9]	validation_0-rmsle:0.50876
[10]	validation_0-rmsle:0.50466
[11]	validation_0-rmsle:0.50041
[12]	validation_0-rmsle:0.49761
[13]	validation_0-rmsle:0.49430
[14]	validation_0-rmsle:0.49138
[15]	validation_0-rmsle:0.48881
[16]	validation_0-rmsle:0.48688
[17]	validation_0-rmsle:0.48510
[18]	validation_0-rmsle:0.48332
[19]	validation_0-rmsle:0.48178
[20]	validation_0-rmsle:0.48022
[21]	validation_0-rmsle:0.47932
[22]	validation_0-rmsle:0.47808
[23]	validation_0-rmsle:0.47685
[24]	validation_0-rmsle:0.47612
[25]	validation_0-rmsle:0.47530
[26]	validation_0-rmsle:0.47466
[27]	validation_0-rmsle:0.47382
[28]	validation_0-rmsle:0.47311
[29]	validation_0-rmsle:0.47262
[30]	validation_0-rmsle:0.47201
[31]	validation_0-rmsle:0.47138
[32]	validation_0-rmsle:0.47090
[33]	validation_0-rmsle:0.47048
[34]	validation_0-rmsle:0.47034
[35]	validation_0-rmsle:0.47017
[36]	validati

is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead


In [7]:
# Save predictions
prediction_df = pd.DataFrame({
    'id': test_df['id'],
    'price_doc': pred
})

prediction_df.to_csv(output_path, index=False)